# My Idea: Barnacle Blob Detection with OpenCV + Contour-Based ML (Model-focused)

We will build a system that takes a cropped image of a tide pool and returns a count of the barnacles by identifying and filtering shapes based on their visual properties like size and circularity. The final output will be both the total count and a visual overlay showing which objects were identified as barnacles.

Use OpenCV to isolate barnacle candidates via blob detection or contours, then train a simple ML classifier (e.g., logistic regression, decision tree) to classify if a blob is a barnacle or not.

## Framework

1. Use color thresholding to isolate the green frame. Crop the interior region.

2. *Image preprocessing* (grayscale, blur, threshold, etc.): Convert the input image into a format that makes it easy to distinguish barnacles from the background.

3. *Contour Detection*: Identify all the potential, distinct objects (shapes) in the pre-processed image. Use OpenCV’s findContours() or SimpleBlobDetector to extract shape features.

4. *Contour Filtering*: Intelligently filter through all the potential objects to discard things that are not barnacles (e.g., too small, too big, not circular enough). This is the "ML" or rule-based intelligence part of the system. From mask1 and mask2, assign labels (1 if contour overlaps mask, 0 otherwise).

5. *Counting & Visualization*: Count the final, filtered contours and draw them on the original image to see what the system actually "counted." Train the ML model using features like area, perimeter, circularity, intensity variance.

Train on _img1.png_ and _img2.png_. Validate using _unseen_img1.png_ and _unseen_img2.png_. Use _mask1.png_ and _mask2.png_, or _masked_img1.png_ and _masked_img2.png_ for contour detection and counting.

## Critical Subtasks

* Identify green wire frame and extract the interior region inside its inner square. Here I used:
- **Input**: _img1.png_, *Output*: cropped image of inner square region
- **Libraries**: OpenCV, NumPy, Math
- **Algos**: Color Space Conversion (cv2.cvtColor), Thresholding (cv2.threshold), Morphological Transformations (cv2.morphologyEx), Edge Detection (cv2.Canny), Line Detection (cv2.HoughLinesP), Line Intersection (Custom Algorithm), Line Merging/Clustering (Custom Algorithm), Perspective Transformation (cv2.getPerspectiveTransform, cv2.warpPerspective)

* Analyze (by counting and visualizing) the mask contours of barnacles. Here I used:
- **Input**: Cropped image of inner square region of _img1.png_, _mask1.png_ or _masked_img1.png_, *Output*: metrics
- **Libraries**: OpenCV, NumPy
- **Algos**: 
*Substack 1* - Image Preprocessing: Grayscale Conversion (cv2.cvtColor), Gaussian Blurring (cv2.GaussianBlur), Adaptive Thresholding (cv2.adaptiveThreshold)
*Substack 2* - Contour Detection: Finding Contours (cv2.findContours)
*Substack 3* - Contour Filtering (The "Intelligence"): Contour Area (cv2.contourArea), Contour Perimeter (cv2.arcLength), Circularity Calculation (Custom Algorithm)
*Substack 4* - Counting & Visualization: Counting (len()), Drawing Contours (cv2.drawContours), Drawing Text (cv2.putText)

## Evaluation:
* Subtask 1: Just check the cropped image
* Subtask 2:
- Count the number of barnacles that the prototype finds in _img1.png_ ```Your_Count```.
- Count the number of barnacles in the ground truth _mask1.png_ ```True_Count```.
- Accuracy is simply how close ```Your_Count``` is to ```True_Count```. For example, ```(Your_Count / True_Count) * 100%```.
- Visual overlay to see false positives/negatives.

## Things I learned:

Even though I have not taken the 70's in CS, thanks to this challenge, I applied for financial aid on Coursera and learned the Supervised Machine Learning: Regression and Classification course. I have also research a lot for the first time regarding OpenCV.

### Import Libraries and Functions

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from barnacles_counter.barnacles_crop import debug_crop, crop_to_inner_frame_hsv, crop_to_inner_frame_canny, crop_to_inner_frame, crop_inner_square
from barnacles_counter.barnacles_analyze import analyze_and_count_barnacles

### Subtask 1: Detect green frame and crop the image

In [ ]:
# Still failed at cropping but was successful at detecting green frame
input_image_file = 'data/img1.png' 
crop_inner_square(input_image_file)

### Subtask 2: Analyze mask contours

In [ ]:
# Run the prototype on the mask1 image
# Use the photographic image as input for the analyzer.
input_image_path = "data/mask1.png"
detected_count, result_image = analyze_and_count_barnacles(input_image_path)

# Display the result
if result_image is not None:
    print(f"Prototype detected {detected_count} barnacles.")
    cv2.imwrite("prototype_output_img1.png", result_image)

# Run the prototype on the masked_img1 image
input_image_path2 = "data/masked_img1.png"
detected_count2, result_image2 = analyze_and_count_barnacles(input_image_path2)

# Display the result
if result_image is not None:
    print(f"Prototype detected {detected_count2} barnacles.")
    cv2.imwrite("prototype_output_img2.png", result_image2)

### My failed attempt on Subtask 1

Regenerate _img1.png_ using Matplot

In [ ]:
img = cv2.imread('data/img1.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb) 
plt.axis('off')
plt.show()


Regenerate masked_img1.png using Matplot

In [ ]:
# Load the “pretty” masked overlay
masked_overlay = cv2.imread('data/masked_img1.png')
# It’s BGR with the barnacle contours already in a color (likely red).
# Convert to RGB for Matplotlib:
masked_overlay_rgb = cv2.cvtColor(masked_overlay, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(6,6))
plt.imshow(masked_overlay_rgb)
plt.axis('off')
plt.show()


Crop green frame using OpenCV's HSV

In [ ]:
crop = crop_to_inner_frame_hsv(img, debug=True)
if crop is None:
    print("Frame not detected—tweak HSV bounds or area threshold.")
else:
    plt.figure(figsize=(6,6))
    plt.imshow(crop)
    plt.axis('off')
    plt.show()

In [ ]:
# load
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# tweak these coordinates until you hit a green wire
y, x = 100, 200  
print("Sample HSV at (x=200,y=100):", hsv[y, x])


In [ ]:
# green thresholds
lower_green = np.array([40, 40, 40])
upper_green = np.array([70, 255, 255])

# Build the mask
green_mask = cv2.inRange(hsv, lower_green, upper_green)

# Show the mask
plt.figure(figsize=(6,6))
plt.imshow(green_mask, cmap='gray')
plt.title("Green‐frame mask")
plt.axis('off')
plt.show()

# Clean it up a bit
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
mask_clean = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN,  kernel)
mask_clean = cv2.morphologyEx(mask_clean,  cv2.MORPH_CLOSE, kernel)

plt.figure(figsize=(6,6))
plt.imshow(mask_clean, cmap='gray')
plt.title("After open + close")
plt.axis('off')
plt.show()

# Quick contour check
cnts, _ = cv2.findContours(mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print("Contours found:", len(cnts))

In [ ]:
debug_crop(img)

Crop green frame using OpenCV's HSV and Canny

In [ ]:
# Run the cropping pipeline
cropped = crop_to_inner_frame(img, debug=True)

if cropped is None:
    print("Frame not detected—tweak thresholds or area cutoff.")
else:
    plt.figure(figsize=(6,6))
    plt.imshow(cropped)
    plt.axis('off')
    plt.show()

Crop green frame using OpenCV's Canny

In [ ]:
# Run the edge-based crop with debug windows
cropped_edge = crop_to_inner_frame_canny(img, debug=True)

if cropped_edge is None:
    print("Frame not found—tweak Canny thresholds or kernel size.")
else:
    # Display the final cropped RGB image inline
    plt.figure(figsize=(6,6))
    plt.imshow(cropped_edge)
    plt.axis('off')
    plt.show()
